## Deliverable 2. Create a Customer Travel Destinations Map.

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from API_key import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file.
path = 'C:/Users/Deva/Desktop/Bootcamp/Repos/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv'
city_data_df = pd.read_csv(path)
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Birao,CF,2020-10-04 03:31:53,10.28,22.79,69.46,92,100,1.90
1,1,Saint-Philippe,RE,2020-10-04 03:31:49,-21.36,55.77,71.60,73,90,2.24
2,2,Ancud,CL,2020-10-04 03:31:56,-41.87,-73.82,41.00,86,0,3.36
3,3,Butaritari,KI,2020-10-04 03:31:39,3.07,172.79,81.50,70,100,7.43
4,4,Harper,LR,2020-10-04 03:32:06,4.38,-7.72,76.35,88,32,8.75


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [36]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,WeatherPy_Database.csv,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Butaritari,KI,2020-10-04 03:31:39,3.07,172.79,81.50,70,100,7.43
4,4,Harper,LR,2020-10-04 03:32:06,4.38,-7.72,76.35,88,32,8.75
7,7,Charters Towers,AU,2020-10-04 03:34:39,-20.10,146.27,84.51,30,28,18.21
11,11,San Patricio,MX,2020-10-04 03:31:45,19.22,-104.70,76.96,82,0,3.49
17,17,Vaini,TO,2020-10-04 03:31:41,-21.20,-175.20,75.20,88,75,12.75


In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database.csv    220
City                      220
Country                   220
Date                      220
Lat                       220
Lng                       220
Max Temp                  220
Humidity                  220
Cloudiness                220
Wind Speed                220
dtype: int64

In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy() #add "Current Description" before lat

# 5b. Create a new column "Hotel Name"
#hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng
3,Butaritari,KI,81.50,3.07,172.79
4,Harper,LR,76.35,4.38,-7.72
7,Charters Towers,AU,84.51,-20.10,146.27
11,San Patricio,MX,76.96,19.22,-104.70
17,Vaini,TO,75.20,-21.20,-175.20
18,Olinda,BR,78.80,-8.01,-34.86
20,Kapaa,US,86.00,22.08,-159.32
24,Saint George,US,78.80,37.10,-113.58
33,Puerto Ayacucho,VE,75.34,5.66,-67.62
44,Banjar,ID,78.80,-8.19,114.97


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [51]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="Vacation_Search")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
#<dt>Current Description</dt><dd>{Current Description}</dd>
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
print(len(locations))

210


In [61]:
# 11a. Add a marker layer for each city to the map.
# Get the maximum temperature.
max_temp = clean_hotel_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=temps,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))